Text Analytics I HWS 23/24

# Home Assignment 1 (35 pts)



Submit your solution via Ilias until 23.59h on Wednesday, October 4th. Late submissions are accepted until 12:00am on the following day, with 1/4 of the total possible points deducted from the score.

Submit your solutions in teams of 3-4 students. Unless explicitly agreed otherwise in advance, **submissions from teams with more or less members will NOT be graded**.
List all members of the team with their student ID in the cell below, and submit only one notebook per team. Only submit a notebook, do not submit the dataset(s) you used. Also, do NOT compress/zip your submission!

You may use the code from the exercises and basic functionalities that are explained in official documentation of Python packages without citing, __all other sources must be cited__. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members may be expelled from the course without warning.

#### General guidelines:
* Make sure that your code is executable, any task for which the code does not directly run on our machine will be graded with 0 points.
* If you use packages that are not available on the default or conda-forge channel, list them below. Also add a link to installation instructions. 
* Ensure that the notebook does not rely on the current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Do not rename any of the datasets you use, and load it from the same directory that your ipynb-notebook is located in, i.e., your working directory.
* Make sure you clean up your code before submission, e.g., properly align your code, and delete every line of code that you do not need anymore, even if you may have experimented with it. Minimize usage of global variables. Avoid reusing variable names multiple times!
* Ensure your code/notebook terminates in reasonable time.
* Feel free to use comments in the code. While we do not require them to get full marks, they may help us in case your code has minor errors.
* For questions that require a textual answer, please do not write the answer as a comment in a code cell, but in a Markdown cell below the code. Always remember to provide sufficient justification for all answers.
* You may create as many additional cells as you want, just make sure that the solutions to the individual tasks can be found near the corresponding assignment.
* If you have any general question regarding the understanding of some task, do not hesitate to post in the student forum in Ilias, so we can clear up such questions for all students in the course.

In [1]:
# studentIDs of all team members
team_members = [12345,67899,880800,234242]

Additional packages (if any):
 - Example: `powerlaw`, https://github.com/jeffalstott/powerlaw

In [1]:
from typing import List, Union, Dict, Set, Tuple
import numpy as np
from numpy.typing import NDArray

### Task 1: Regular expressions (9 pts)
In this task you are asked to create regular expressions that meet the specified conditions. Please use the ``re`` package for the following subtasks.

In [2]:
# import the re package 
import re

__a)__ Write a regular expression that returns all integer numbers from a text that are surrounded by whitespaces. __(1 pt)__ 

In [3]:
# your code here

s1 = "hello 9 jdie2324 di 3 dewd 200"

result = re.findall(r'\s(\d+)\s', s1)

print(result)

['9', '3']


__b)__ Write a regular expression that returns all valid years that are surrounded by whitespaces in a text. A valid year is a 4 digit number in the range from 0000 to 2022. __(2 pts)__  
You do not need to account for overlaps, e.g., `' 2023 2024 ' => [2023]` is ok. Bonus points if you do so (`' 2023 2024 ' => [2023,2024]`). __(+2 pts)__

In [4]:
# your code here

s1 = " 0000 1110 1234 1998 "

result = re.findall(r'\b([0-1]\d{3}|20[0-1]\d{1}|202[0-2])\b', s1) # -> wrong #0000~1999 #2000~2019 #2020~2022

print(result)

['0000', '1110', '1234', '1998']


__c)__ Write a regular expression that returns all dates in the format YYYY-MM-DD or YYYY/MM/DD from a given text. Make sure that YYYY is a valid year (see task __b)__), MM is a valid month (01 to 12) and DD is a valid day (01 to 31).
There is no need to make sure that e.g. XXXX-02-31 does not exist.  __(2 pts)__

In [5]:
# your code here

s1 = "2022-05-12 and 2012/05/31"

patterns = re.findall(r'(\b([0-1]\d{3}|20[0-1]\d{1}|202[0-2])[-/](0[1-9]|1[0-2])[-/](0[1-9]|[12][0-9]|3[01])\b)', s1)
patterns_list = [pattern[0] for pattern in patterns]

print(patterns_list)

['2022-05-12', '2012/05/31']


__d)__ Assume you are given a list ``l`` of strings like the one below. Using regular expressions, return a list that contains all elements from ``l`` that don't contain both, the letter ``a`` and ``e`` and store the result in a variable ``l_filtered``.  __(2 pts)__

__Example:__ _given the list_  

``l = ["apple", "cucumber", "tomato", "zucchini", "pumpkin", "pear", "raspberry", "blueberry"]``  

_you should return_  

``l_filtered = ['cucumber', 'tomato', 'zucchini', 'pumpkin', 'blueberry']``.

In [6]:
# example list
l = ["apple", "cucumber", "tomato", "zucchini", "pumpkin", "pear", "raspberry", "blueberry"]

In [7]:
# your code here

l_filtered = [w for w in l if not re.search(r'\b(?=\w*a)(?=\w*e)\w+\b', w)] #\w+ -> matches one or more word characters, ensuring that the word contains both 'a' and 'e'

print(l_filtered)

['cucumber', 'tomato', 'zucchini', 'pumpkin', 'blueberry']


__e)__ For the given string ``s`` with 4 lines, change the _whole_ word ``pot`` (i.e. ``pottery`` should not be changed) to ``1234`` only if it is at the start of a line.  __(2 pts)__

In [8]:
s = '''\
pottery clot pot 
pot dot plot hot
spot rot pot got
not shot forgot'''

In [9]:
# your code here

lines_list = s.split('\n')
result = re.sub(r'^pot\b', '1234', s, flags=re.MULTILINE)

print(result)

pottery clot pot 
1234 dot plot hot
spot rot pot got
not shot forgot


### Task 2: Finding the most similar word (18 pts)
The goal of this task is, given a corpus, to find the most similar word for a provided word. As an example we will consider the King James Bible that is is included in the ``gutenberg`` corpus and we are looking to find the word that is most similar to ``god``. We consider two words similar if they appear in the same word context.

__a) Cleaning the input (3 pts)__

Write a function that, given a list of tokens, returns a list of tokens that we consider valid for our task. Moreover, all input tokens should be converted to lower case.

An *invalid* token is a token that 
- is a punctuation mark (consider `string.punctuation`).
- is entirely numeric digits (e.g. `"123"`)
- optionally, if `remove_stopwords=True` then stopwords in the English language are also not considered valid tokens (use nltk's stopwords). 

Use the function signature specified in the cell below.

In [10]:
import nltk
from nltk.corpus.reader.util import StreamBackedCorpusView 

import string
from typing import List, Union
from nltk.corpus import stopwords

nltk.download('stopwords')

def get_valid_tokens(tokens: Union[List[str], StreamBackedCorpusView], remove_stopwords: bool=False) -> List[str]:
    """
    :param tokens: list of tokens that should be cleaned
    :param remove_stopwords: bool indicating if stopwords should be removed 
                             False by default
    :return: list of valid tokens
    """
    # your code here

    # Convert input tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Define a set of stopwords if remove_stopwords is True
    if remove_stopwords:
        stop_words = set(stopwords.words('english')) 
    else:
        stop_words = set()

    valid_tokens = []

    for token in tokens:
        # Check if the token is entirely numeric
        if token.isnumeric():
            continue  # Skip numeric tokens

        # Check if the token is a punctuation mark
        if all(char in string.punctuation for char in token):
            continue  # Skip punctuation tokens

        # Check if the token is a stopword (if remove_stopwords is True)
        if remove_stopwords and token in stop_words:
            continue  # Skip stopwords

        # If the token passed all checks, it's valid
        valid_tokens.append(token)

    return valid_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yinhsuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


__b) Counting the surroundings (6 pts)__

In our simple model of word similarity we consider words as similar if they are being used in the same context (i.e. they have similar words surrounding them). 

Implement a function that, given a list of words, returns the count of all words in a designated word's surrounding. The context size indicates how many words to the left and right we consider, i.e. a context size of 1 indicates that we only consider the words immediately before and after a central word to be in its context. 

Your function should return a dictionary which maps each word $w$ from the input list to a dictionary. The inner dictionary should map each word that appears in the context of the central word $w$ to a number that indicates how often it appears in the context of $w$.

Make sure your implementation has linear complexity in the vocabulary / corpus length. Use the function signature specified in the cell below.

__Hint:__ Instead of the inner dictionary you can alternatively use `Counter` or `defaultdict` which can be found in the Python module `collections`. Moreover, consider the function ``ngrams`` from ``nltk``.

__Example:__ _For the input_
`['hi', 'james', 'how', 'are', 'you', 'hello', 'catherine', 'how', 'are', 'you']` _and_ `context_size=1`
_we wish to obtain:_
```
{'hi': {'james': 1},
 'james': {'hi': 1, 'how': 1},
 'how': {'james': 1, 'are': 2, 'catherine': 1},
 'are': {'how': 2, 'you': 2},
 'you': {'are': 2, 'hello': 1},
 'hello': {'you': 1, 'catherine': 1},
 'catherine': {'hello': 1, 'how': 1}}
```
__Explanation:__ _The word_ `hi` _is only surrounded by_ `james`_. The word_ `james` _is surrounded by_ `hi` _and_ `how` _. The word_ `how` _is surrounded by_ `james`_, by_ `catherine` _and by_ `are` _twice, ..._

_For_ `contextsize=2` _we obtain:_
```
{'hi': {'james': 1, 'how': 1},
'james': {'hi': 1, 'how': 1, 'are': 1},
'how': {'are': 2, 'you': 2, 'hi': 1, 'james': 1, 'hello': 1, 'catherine': 1},
'are': {'how': 2, 'you': 2, 'james': 1, 'hello': 1, 'catherine': 1},
'you': {'how': 2, 'are': 2, 'hello': 1, 'catherine': 1},
'hello': {'are': 1, 'you': 1, 'catherine': 1, 'how': 1},
'catherine': {'you': 1, 'hello': 1, 'how': 1, 'are': 1}}
```

In [11]:
from typing import List, Dict, Union
from collections import defaultdict
from nltk.util import ngrams
from nltk.corpus.reader.util import StreamBackedCorpusView
import json

def get_surrounding_counts(tokens: Union[List[str], StreamBackedCorpusView], context_size: int) -> Dict[str, Dict[str, int]]:
    """
    :param tokens: list of tokens
    :param context_size: integer that indicates the number of context words that are considered on both sides of the central word
    :return: dict of dicts that holds the count of context words for each input token
    """
    # your code here
    
    context_counts = {}
    
    for i, token in enumerate(tokens):
        left_bound = max(i - context_size, 0)
        right_bound = min(i + context_size + 1, len(tokens))
        context = tokens[left_bound:i] + tokens[i+1:right_bound] #context: words at left side and right side

        if token not in context_counts:
            context_counts[token] = {}
        
        token_dict = context_counts[token]

        for word in context:
            if word in token_dict:
                token_dict[word] += 1
            else:
                token_dict[word] = 1
    
    return context_counts

__c) Keeping the top $k$ words in context (2 pts)__

To reduce the size of our result from task __b)__, we will only consider the most frequent context words for each token. Therefore, write a function that keeps only the $k$ most frequent words in the context of a designated word. Ties are resolved in lexicographic order (e.g. _**A**braham_ would be preferred to _**B**ethlehem_). The function should return a dictionary that maps each word in the outer dictionary to a __set__ of their top $k$ most frequent context words.

In [12]:
def to_sets(context_dict: Dict[str, Dict[str, int]], k: int) -> Dict[str, Set[str]]:
    """
    :param context_dict: dict of dicts that holds the count of context words for each word
    :param k: integer that specifies how many context words should be kept
    :return: dict that maps each word to a set of its k most frequent context words
    """
    # your code here
    
    result = {}
    
    for word, context_counts in context_dict.items():
        # Sort context words by frequency in descending order and then lexicographically
        sorted_context = sorted(context_counts.keys(), key=lambda x: (-context_counts[x], x))
        
        # Select the top k context words
        top_k_context = sorted_context[:k]
        
        # Store the selected context words in a set
        result[word] = set(top_k_context)
    
    return result

__d) Finding the most similar words (4 pts)__

Given a dictionary as obtained in task __c)__ and a word $w$, we want to find the words that have the highest similarity to $w$ in terms of their context. We operationalize context similarity with the Jaccard index (https://en.wikipedia.org/wiki/Jaccard_index).
The Jaccard index of two sets $A$ and $B$ (in our case the sets of context words) is defined as:

$J(A, B) = \frac{|A \cap B|}{|A \cup B|}$

Write a function that returns the words that have the highest similarity to an input word $w$ (ignoring the input word itself). Since several words may have the same Jaccard similarity to the input word, your function should return the set of words that are most similar to $w$ and the respective value of the Jaccard similarity. Use the function signature specified in the cell below.

In [13]:
def find_most_similar_words(input_word: str, contexts: Dict[str, Set[str]]) -> (Set[str], float):
    """
    :param input_word: string that represents the word we are interested in
    :param contexts: dict that maps each word to a set of its most frequent context words
    :returns: 
        - set of the most similar words to the input word
        - float that indicates the highest Jaccard similarity to the input word
    """
    # your code here
    
    most_similar_words = set()  # Initialize an empty set to store the most similar words
    highest_similarity = 0.0  # Initialize the highest Jaccard similarity
    
    # Get the context set for the input word
    input_context = contexts.get(input_word, set())
    
    for word, context_set in contexts.items():
        if word != input_word:  # Exclude the input word itself
            # Calculate Jaccard similarity
            intersection = len(input_context.intersection(context_set))
            union = len(input_context.union(context_set))
            similarity = intersection / union
            
            # Update most similar words and highest similarity
            if similarity > highest_similarity:
                most_similar_words = {word}
                highest_similarity = similarity
            elif similarity == highest_similarity:
                most_similar_words.add(word)
    
    return most_similar_words, highest_similarity

__e) Bringing it all together (3 pts)__

Finally, we want to apply our functions to the King James Bible (`'bible-kjv.txt'`) that is part of the ``gutenberg`` corpus. We intend to find the word(s) that is (are) most similar to ``god``. Follow the steps below:

i) Obtain a list of all tokens from the King James Bible and store it in a variable ``tokens``.  

ii) Clean the list of tokens with your function from __a)__ to get the list of valid tokens (without removing stopwords) and store it in a variable ``valid_tokens``.  

iii) Apply your function from __b)__ to count the context words for all valid tokens with a ``context_size`` of 2 and store the result in a variable ``context_counts``.  

iv) Using your function from __c)__, keep only the 20 most frequent words in a valid tokens context and store the result in a variable ``context_words``.  

v) Finally, find the most similar words to the word _god_ with your function from __d)__ and store the set of most similar words in the variable ``set_god`` and the highest Jaccard similarity in the variable ``sim_god``.  

In [14]:
# your code here

from nltk.corpus import gutenberg
from typing import List

# ----------------- i -----------------
# Load the King James Bible
file_id = 'bible-kjv.txt'
tokens = gutenberg.words(file_id)

# ----------------- ii -----------------
# Call the get_valid_tokens function to obtain valid tokens
valid_tokens = get_valid_tokens(tokens)

# Print the first few valid tokens for inspection
# print("Valid tokens:", valid_tokens[:10])

# ----------------- iii -----------------
context_size = 2
context_counts = get_surrounding_counts(valid_tokens, context_size)

# ----------------- iv -----------------
k = 20
context_words = to_sets(context_counts, k)

# ----------------- v -----------------
input_word = "god"
set_god, sim_god = find_most_similar_words(input_word, context_words)

# Check if "god" was in the context_words dictionary
if input_word not in context_words:
    print(f"'{input_word}' was not found in the context_words dictionary.")

# Print the most similar words and the highest Jaccard similarity
print(f"Most similar words to '{input_word}': {set_god}")
print(f"Highest Jaccard similarity: {sim_god}")

Most similar words to 'god': {'father', 'lord'}
Highest Jaccard similarity: 0.6


### Task 3: Minimum cost string alignment (8 pts)

In this tak we want to compute an alignment between two strings, that has minimum edit distance. 

Implement a function that takes two strings and their edit distance matrix and returns a minimum cost alignment. You can assume that the edit distance matrix is provided by the function that you implemented in exercise 3, task 2, with a substitution cost of 2. 

A minimum cost alignment consists of two strings that, printed below each other comprise the alignment, where insertions and deletions are represented by a ``*``. Use the function signature in the cell below.

__Example:__ _Given the input strings_ ``"INTENTION"`` _and_ ``"EXECUTION"`` _and the corresponding edit distance matrix:_

``[[ 0  1  2  3  4  5  6  7  8  9]
 [ 1  2  3  4  5  6  7  6  7  8]
 [ 2  3  4  5  6  7  8  7  8  7]
 [ 3  4  5  6  7  8  7  8  9  8]
 [ 4  3  4  5  6  7  8  9 10  9]
 [ 5  4  5  6  7  8  9 10 11 10]
 [ 6  5  6  7  8  9  8  9 10 11]
 [ 7  6  7  8  9 10  9  8  9 10]
 [ 8  7  8  9 10 11 10  9  8  9]
 [ 9  8  9 10 11 12 11 10  9  8]]``,
  
_your function should return the two strings_ ``INTE***NTION`` _and_ ``***EXECUTION`` _that represent the alignment, when printed below each other:_
 
 ``INTE***NTION``    
 ``***EXECUTION`` 
 
 __Remark:__ _The alignment in the example above is not the only solution. In this task all alignments with minimum edit distance are accepted._


In [15]:
def edit_distance(str1: str, str2: str, cost_of_substitute: int=2) -> (int, NDArray[NDArray[int]]):
    '''
    :param str1: source string 
    :param str2: target string
    :cost_of_substitute: cost for substituting mismatching letters 
    :returns:
        - minimum edit distance of str1 and str2
        - minimum edit distance table
    '''
    I = len(str1) + 1  # +1 for hashtag symbol (empty string)
    J = len(str2) + 1
    D = np.zeros((I,J), dtype=int)
    
    # initialize first row and column
    D[:,0] = np.arange(I)  # index 0 of all elements (first column)
    D[0,:] = np.arange(J)  # all entries of element at index 0 (first row)
    
    for i in range(1,I):  # loop through rows
        for j in range(1,J):  # loop through columns
            sub = D[i-1, j-1] # init cost same letters
            if str1[i-1] != str2[j-1]:
                  sub += cost_of_substitute # not same letters
            
            D[i,j] = min(D[i-1,j] + 1, D[i,j-1] + 1, sub)  # new value
            
    return D[I-1,J-1], D

def get_alignment(str1: str, str2: str, D: NDArray[NDArray[int]]) -> (str, str): 
    '''
    :param str1: first string for alignment
    :param str2: second string for alignment
    :param D: edit distance matrix of str1 and str2
    :returns: tuple of strings that indicate the alignment of the input strings
    '''
    # your code here

    i, j = len(str1), len(str2)
    alignment_str1 = ""
    alignment_str2 = ""

    while i > 0 or j > 0:
        if i > 0 and j > 0 and str1[i - 1] == str2[j - 1]:
            # No operation, characters are the same
            alignment_str1 = str1[i - 1] + alignment_str1
            alignment_str2 = str2[j - 1] + alignment_str2
            i -= 1
            j -= 1
        elif i > 0 and D[i][j] == D[i - 1][j] + 1:
            # Deletion
            alignment_str1 = str1[i - 1] + alignment_str1
            alignment_str2 = "*" + alignment_str2
            i -= 1
        elif j > 0 and D[i][j] == D[i][j - 1] + 1:
            # Insertion
            alignment_str1 = "*" + alignment_str1
            alignment_str2 = str2[j - 1] + alignment_str2
            j -= 1
        else:
            # Substitution
            alignment_str1 = str1[i - 1] + alignment_str1
            alignment_str2 = str2[j - 1] + alignment_str2
            i -= 1
            j -= 1

    # Add asterisks to the beginning of alignment_str1 and alignment_str2 to match the example format
    alignment_str1 = "*" * i + alignment_str1
    alignment_str2 = "*" * j + alignment_str2

    return alignment_str1, alignment_str2

def find_minimum_alignment(str1: str, str2: str, cost_of_substitute: int=2):
    cost, D = edit_distance(str1, str2, cost_of_substitute)
    alignment_str1, alignment_str2 = get_alignment(str1, str2, D)
    return alignment_str1, alignment_str2

str1 = "INTENTION"
str2 = "EXECUTION"
alignment_str1, alignment_str2 = find_minimum_alignment(str1, str2)
print(alignment_str1)
print(alignment_str2)

**INTE**NTION
EX***ECU*TION
